In [12]:
import re
import time
import os
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from string import Template
import urllib
import json
import pickle


from pprint import pprint
import sys

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,davies_bouldin_score,calinski_harabasz_score
from sklearn.decomposition import PCA, SparsePCA
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [13]:
pd.set_option('display.max_rows', 500)


In [14]:
## 經濟地理資訊平台API抓的資料

with open('/content/drive/MyDrive/03_RESOURCE/GIS/gis_open_api.pickle', 'rb') as f:
    api_data = pickle.load(f)

## 對照表整併

In [15]:

# 分行爬蟲資訊

## 各分行地址與基本資訊
brn_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/taishin_brn.csv').drop_duplicates()

## 分行地址與點位
brn_xy_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/brn_xy_df.csv')

## 合併並去掉重複
brn_df = brn_df.merge(brn_xy_df,how='left' ,on = 'addr').copy()
brn_df = brn_df.drop_duplicates()

## 作為主表
brn_code_df = brn_df[['brn', 'addr', 'lon', 'lat']]

## 整理經濟三級發布區
brn_code3_df = pd.DataFrame(
    [(x,api_data[x]['code3_data']['ADMIV']['CODE3']) for x in list(api_data.keys()) if x !='南崁分行'],
    columns = ['brn','eco_code3']
)
brn_code3_df = pd.concat([brn_code3_df, pd.DataFrame([['南崁分行','A6800005028']], columns=['brn','eco_code3'])])
brn_code3_df = brn_code3_df.reset_index(drop=True)

## 整理統計二級發布區
brn_code2_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/分行統計區代碼對照.csv', encoding='CP950')
brn_code2_df = brn_code2_df[['ID','縣市', '鄉鎮市區', '村里', '二級發布區', '一級發布區', '最小統計區']].rename(
    {
      'ID': 'brn',
      '縣市':'city',
      '鄉鎮市區':'town',
      '村里':'village',
      '二級發布區':'code_2',
      '一級發布區':'code_1',
      '最小統計區':'code_min'
    },
    axis=1
)

# 整併
brn_code_df = brn_code_df.merge(brn_code2_df, how='left', on=['brn'])
brn_code_df = brn_code_df.merge(brn_code3_df, how='left', on=['brn'])

# 資料清理
## 處理異體字跟鄉鎮市區層級更名

brn_code_df['town'] = brn_code_df['town'].replace('員林鎮', '員林市')
brn_code_df.loc[brn_code_df['town']=='新店區','village'] = brn_code_df.loc[brn_code_df['town']=='新店區','village'].replace('五?里', '五峰里')


## 電信信令人口資料

In [16]:

## 電信信令人口資料
tele_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/109年11月行政區電信信令人口統計資料_鄉鎮市區.csv', encoding='CP950')

In [17]:
tele_df

,COUNTY_ID,COUNTY,TOWN_ID,TOWN,NIGHT_WORK,DAY_WORK(7:00~13:00),DAY_WORK(13:00~19:00),DAY_WORK,NIGHT_WEEKEND,DAY_WEEKEND(7:00~13:00),DAY_WEEKEND(13:00~19:00),DAY_WEEKEND,MORNING_WORK,MIDDAY_WORK,AFTERNOON_WORK,EVENING_WORK,MORNING_WEEKEND,MIDDAY_WEEKEND,AFTERNOON_WEEKEND,EVENING_WEEKEND,INFO_TIME
0,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,平日夜間停留人數,平日上午活動人數,平日下午活動人數,平日日間活動人數,假日夜間停留人數,假日上午活動人數,假日下午活動人數,假日日間活動人數,平日早晨旅次,平日中午旅次,平日午後旅次,平日晚上旅次,假日早晨旅次,假日中午旅次,假日午後旅次,假日晚上旅次,資料時間
1,65000,新北市,65000010,板橋區,577007,468604,459821,461811,574620,553665,547852,552865,1370756.45,1199378.12,1217279.46,1417770.61,1181447.57,1447237.98,1452579.49,1427165.52,109Y11M
2,65000,新北市,65000020,三重區,426580,353762,336820,343466,424259,402368,379533,395318,1047368.37,873343.72,880980.15,1000009.8,887589.65,1004799.36,975547.52,939548.09,109Y11M
3,65000,新北市,65000030,中和區,470287,414013,398795,408358,466276,439917,417541,433865,1142144.94,995924.74,991048.66,1074345.45,966434.32,1062291.77,1036495.62,972990.59,109Y11M
4,65000,新北市,65000040,永和區,232342,169268,157426,161573,230134,215493,199485,210496,513518.18,415321.92,414020.3,495365.41,461549.08,526167.13,495412.61,485350.57,109Y11M
5,65000,新北市,65000050,新莊區,446481,381702,374563,377239,443620,426704,419179,426607,1093803.05,925342.9,929744.79,1077273.18,925176.4,1086124.76,1056036.59,1001453.5,109Y11M
6,65000,新北市,65000060,新店區,346150,316147,303348,309344,343603,332771,315198,328460,780546.29,694581.09,693588.67,718829.75,650811.86,746315.92,729628.39,657317.14,109Y11M
7,65000,新北市,65000070,樹林區,202270,190785,187977,189656,202721,195384,190928,195311,489739.83,420652.53,426735.44,451024.04,397929.01,447903.69,441277.45,430214.61,109Y11M
8,65000,新北市,65000080,鶯歌區,90900,81480,80218,80939,91143,87876,86757,87625,207558.07,171225.54,179422.75,188227.64,175110.32,203134.32,207767.99,192814.35,109Y11M
9,65000,新北市,65000090,三峽區,120827,105834,104205,104524,121910,123685,120884,122589,258058.78,234151.71,238437.49,244033.42,238756.25,304760.89,301355.44,253190.16,109Y11M


## 周圍公司行號

In [18]:
# 公司行號工廠資訊

poi_num_df = []
for brn in list(api_data.keys()):
    df = pd.DataFrame([list(api_data[brn]['around_num'].values())],columns=['factory_num','bu_num', 'cmp_num', 'stk_num'])
    poi_num_df.append(df)
poi_num_df = pd.concat(poi_num_df)
poi_num_df['brn'] = list(api_data.keys())

In [19]:
%%capture
'''
## openstreetmap資料
import pickle
with open('/content/drive/MyDrive/03_RESOURCE/GIS/brn_poi.pickle', 'rb') as f:
  poi = pickle.load(f)
'''

## 人口

In [20]:
## 人口資料

pop_ratio = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口指標.csv')
pop_data = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口資料.csv')

pop_ratio = pop_ratio.rename({'CODE2':'code_2'},axis=1)
pop_data = pop_data.rename({'CODE2':'code_2'},axis=1)


## 年收

In [21]:
## 年收入資料 
## 資料年度107年 單位千元
sal_df = pd.read_csv('https://www.fia.gov.tw/WEB/fia/ias/ias106/106_165-9.csv')

## 電子發票消費熱度

In [22]:
## 政府製作的消費熱度

e_inv_ratio = pd.read_csv('https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/1/1124193D-09F5-4711-AB9A-01848E3B88E4/0/?fileType=csv')

In [23]:
## 刪除所有字串內空白

def whitespace_remover(dataframe):
    # iterating over the columns
    for i in dataframe.columns:        
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':       
            # applying strip function on column
            dataframe[i] = dataframe[i].str.replace('\s+', '', regex=True)
        else:              
            # if condn. is False then it will do nothing.
            pass

In [24]:
e_inv_ratio = e_inv_ratio[e_inv_ratio['年度']==2020][['縣市','鄉鎮市區', '村里','主行業別', '消費熱度計算來源', '張數指標', '銷售額指標']]
e_inv_ratio.eval('綜合指標 = (張數指標+銷售額指標)/2', inplace=True)

## groupby mean計算不同計算來源

e_inv_ratio = e_inv_ratio.groupby(['縣市', '鄉鎮市區', '村里', '主行業別'])[['綜合指標']]\
.agg('mean')\
.reset_index(level=[0,1,2,3])

e_inv_ratio = e_inv_ratio.pivot_table(index=['縣市', '鄉鎮市區', '村里'], columns = ['主行業別'], values = ['綜合指標']).fillna(0)
e_inv_ratio.columns = ['_'.join(col) for col in e_inv_ratio.columns.values]
e_inv_ratio = e_inv_ratio.reset_index()


In [25]:
whitespace_remover(e_inv_ratio)

In [26]:
e_inv_ratio.columns = ['city','town','village','hotel_ind', 'retail_ind', 'ctring_ind']

In [27]:
e_inv_ratio['village'] = e_inv_ratio['village'].replace('羣賢里','群賢里')
#e_inv_ratio.query('village.str.contains("賢")', engine='python')

電子發票消費熱度指標

https://data.gov.tw/dataset/36843

https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/1/1124193D-09F5-4711-AB9A-01848E3B88E4/0/?fileType=csv

finlab實價登錄爬蟲
https://www.finlab.tw/real-estate-analasys-histograms/

## 開始整併

In [28]:
'''
性比例	戶量	人口密度  扶養比	扶幼比	扶老比	老化指數
'''
pop_ratio.head(3)

,city,code_2,M_F_RAT,P_H_CNT,P_DEN,DEPENDENCY_RAT,A0A14_A15A65_RAT,A65UP_A15A64_RAT,A65_A0A14_RAT,INFO_TIME
0,南投縣,A0801-01,91.86,2.91,3512.41,52.33,27.77,24.56,88.45,109Y12M
1,南投縣,A0801-02,80.50,2.23,1568.15,94.48,20.04,74.44,371.56,109Y12M
2,南投縣,A0801-03,115.34,2.73,384.38,40.75,13.08,27.66,211.48,109Y12M


In [29]:
sal_dict={
  '縣市':'city',
  '鄉鎮市區':'town',
  '村里':'village',
  '平均數':'sal_mean',
  '中位數':'sal_med'
}

sal_df.columns = ['city', 'town', 'village', 'tax_unit_cnt', 'all_amt', 'sal_mean', 'sal_med', 'Q1', 'Q3', 'std', 's']
sal_df['village'] = sal_df['village'].replace('羣賢里','群賢里')

In [30]:
## 人口指標
brn_gis_df = brn_code_df.merge(pop_ratio[['code_2','P_DEN','A65UP_A15A64_RAT']], how='left', left_on='code_2', right_on='code_2')

## 年收
brn_gis_df = brn_gis_df.merge(sal_df[['city', 'town', 'village','sal_mean', 'sal_med']], how='left', on=['town','city','village'])

## 消費熱度
brn_gis_df = brn_gis_df.merge(e_inv_ratio, how='left', on=['town','city','village'])


## 公司行號工廠
brn_gis_df = brn_gis_df.merge(poi_num_df, how='left', on='brn')



In [68]:
feat_list = [
    'P_DEN', 'A65UP_A15A64_RAT',
    'sal_mean',  
    'factory_num', 'bu_num', 'cmp_num', 'stk_num', 
    'hotel_ind', 'retail_ind', 'ctring_ind'
    #先排除電子發票
]

In [69]:
arr = brn_gis_df[feat_list]
brn_gis_df[['brn','city','town','village']+feat_list].head()

,brn,city,town,village,P_DEN,A65UP_A15A64_RAT,sal_mean,factory_num,bu_num,cmp_num,stk_num,hotel_ind,retail_ind,ctring_ind
0,營業部(總行),臺北市,中山區,民安里,19921.56,34.61,1290,0,1369,5066,15,60.0,99.00,99.00
1,敦南分行,臺北市,大安區,敦安里,41990.78,34.41,2085,0,829,4887,11,73.5,89.75,80.00
2,新生分行,臺北市,中正區,幸市里,27008.20,36.09,2241,1,399,2574,14,0.0,92.00,91.75
3,新莊分行,新北市,新莊區,中華里,60972.12,15.98,1021,6,1793,1122,0,0.0,63.00,64.75
4,桃園分行,桃園市,桃園區,文明里,15269.66,28.40,915,19,656,637,1,0.0,55.50,60.75


In [70]:
brn_gis_df[brn_gis_df.isnull().any(axis=1)]

,brn,addr,lon,lat,city,town,village,code_2,code_1,code_min,eco_code3,P_DEN,A65UP_A15A64_RAT,sal_mean,sal_med,hotel_ind,retail_ind,ctring_ind,factory_num,bu_num,cmp_num,stk_num


## EDA

[觀光景點消費熱度分析-電子發票載具客源地區統計-資料集](https://sip.einvoice.nat.gov.tw/ods-main/ODS303E/691C0280-CEFB-488F-9E71-6AA4F39A41CD/30/Mjs=?FUNCTION_ID=ODS303E&BUILD_INFO=20211008-1333&SYSTEM_ID=ODS&SYSTEM_NAME=%E9%9A%A8%E9%81%B8&ENVIRONMENT_DISPLAY_NAME=&TITLE=%E6%AD%A1%E8%BF%8E%E8%92%9E%E8%87%A8+%E8%B2%A1%E6%94%BF%E9%83%A8%E9%9B%BB%E5%AD%90%E7%99%BC%E7%A5%A8+%E6%99%BA%E6%85%A7%E5%A5%BD%E7%94%9F%E6%B4%BB+%E6%9C%8D%E5%8B%99%E5%B9%B3%E5%8F%B0)

[電信信令人口統計之建置、分析與應用](https://ws.moi.gov.tw/Download.ashx?u=LzAwMS9VcGxvYWQvNDAwL3JlbGZpbGUvMC8xNDk0NS85NzMxZjkxNi01MzU5LTQzZDktYmVlOS0zNjMyYTUwOTcxMDYucGRm&n=6Zu75L%2Bh5L%2Bh5Luk5Lq65Y%2Bj57Wx6KiI5LmL5bu6572u44CB5YiG5p6Q6IiH5oeJ55SoLnBkZg%3D%3D&icon=..pdf)

In [84]:
fnl_df = brn_gis_df[['brn','lon','lat','city','town','village']+feat_list]

In [85]:
fnl_df.insert(0,'lon_lat', brn_gis_df['lat'].astype(str)+','+brn_gis_df['lon'].astype(str))

In [86]:
clstr_feat = [
  'P_DEN',
  'A65UP_A15A64_RAT',
  'sal_mean',
  'factory_num',
  'bu_num',
  'cmp_num',
  'stk_num',
  'hotel_ind',
  'retail_ind',
  'ctring_ind'
]

In [87]:
from sklearn.preprocessing import StandardScaler

In [88]:
clstr_arr = StandardScaler().fit_transform(fnl_df[clstr_feat])

In [89]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [90]:
#clustering = AgglomerativeClustering(n_clusters=5).fit(clstr_arr)
clustering = KMeans(n_clusters=5, max_iter=5000).fit(clstr_arr)


In [91]:
clustering.labels_

array([2, 2, 2, 0, 1, 3, 0, 3, 1, 2, 3, 2, 2, 1, 3, 0, 3, 2, 0, 1, 3, 1,
       3, 1, 2, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 1, 1, 3, 0, 2, 0, 3, 0, 3,
       4, 2, 1, 3, 3, 3, 0, 0, 2, 2, 2, 2, 1, 3, 3, 2, 1, 3, 3, 1, 2, 1,
       3, 3, 0, 3, 0, 3, 3, 2, 3, 4, 3, 3, 3, 2, 1, 1, 3, 0, 3, 0, 0, 1,
       2, 3, 1, 3, 1, 1, 3, 1, 1, 3, 1, 3, 0, 1, 3, 3], dtype=int32)

In [92]:
fnl_df.insert(0,'clstr', clustering.labels_)

In [93]:
for i in range(0,5):
  print('群'+str(i))
  print(fnl_df[fnl_df['clstr']==i]['brn'].unique())

群0
['新莊分行' '板橋分行' '天母分行' '中和分行' '古亭分行' '民生分行' '和平分行' '大直分行' '南門分行' '延平分行'
 '景平分行' '北師分行' '江翠分行' '松德分行' '文山分行']
群1
['桃園分行' '台南分行' '苓雅分行' '嘉義分行' '花蓮分行' '七賢分行' '三重分行' '金華分行' '彰化分行' '五甲分行'
 '永和分行' '板南分行' '三和分行' '東高雄分行' '新店分行' '淡水分行' '東基隆分行' '北大分行' '竹北分行' '八德分行'
 '員林分行' '右昌分行' '沙鹿分行' '羅東分行' '副都心分行']
群2
['營業部(總行)' '敦南分行' '新生分行' '南京東路分行' '信託部' '國外部' '國際金融業務分行' '信義分行' '建橋分行'
 '內湖分行' '西門分行' '敦北分行' '忠孝分行' '復興分行' '建北分行' '基隆路分行' '松江分行' '南松山分行' '南港分行']
群3
['台中分行' '高雄分行' '中壢分行' '蘆洲分行' '大里分行' '豐原分行' '新竹分行' '北台中分行' '永福分行' '屏東分行'
 '大安分行' '龍潭分行' '崇德分行' '後甲分行' '海佃分行' '鳳山分行' '太平分行' '北高雄分行' '逢甲分行' '竹科分行'
 '南屯分行' '民權分行' '三民分行' '北新店分行' '府城分行' '北桃園分行' '大墩分行' '南崁分行' '文心分行' '岡山分行'
 '大雅分行' '成功分行' '石牌分行' '永康分行' '新板分行' '永華分行' '關東橋分行' '南寮分行' '市府分行' '竹南分行'
 '東湖分行' '景美分行' '雄科分行']
群4
['南新莊分行' '汐止分行']


- 群1

- 群2
- 群3
- 群4

In [94]:
fnl_df.groupby('clstr').describe().T#.to_csv('brn_gis.csv')

clstr                              0             1  ...             3             4
lon              count     15.000000     25.000000  ...     43.000000      2.000000
                 mean     121.520136    120.967480  ...    120.777207    121.525292
                 std        0.038307      0.573170  ...      0.459723      0.150317
                 min      121.452717    120.191417  ...    120.179960    121.419001
                 25%      121.503275    120.329075  ...    120.324744    121.472146
                 50%      121.524970    121.011425  ...    120.670190    121.525292
                 75%      121.547572    121.482657  ...    121.120587    121.578437
                 max      121.575038    121.769916  ...    121.606744    121.631582
lat              count     15.000000     25.000000  ...     43.000000      2.000000
                 mean      25.035555     24.146601  ...     24.036121     25.045867
                 std        0.031449      0.996555  ...      0.912002      0.027726
                 min       24.988717     22.593992  ...     22.605142     25.026262
                 25%       25.020275     22.998680  ...     22.994295     25.036065
                 50%       25.029391     24.678296  ...     24.161794     25.045867
                 75%       25.052134     25.007434  ...     24.853043     25.055670
                 max       25.104983     25.170657  ...     25.116466     25.065473
P_DEN            count     15.000000     25.000000  ...     43.000000      2.000000
                 mean   58170.079333  23416.907200  ...  20309.486744  43800.685000
                 std    16859.398729  13493.845220  ...  11694.657320    889.872671
                 min    31204.040000   5294.260000  ...    165.800000  43171.450000
                 25%    44251.720000  14292.880000  ...  12990.970000  43486.067500
                 50%    60972.120000  19976.470000  ...  17124.480000  43800.685000
                 75%    69055.275000  31752.880000  ...  26488.610000  44115.302500
                 max    89039.670000  56098.040000  ...  56747.290000  44429.920000
A65UP_A15A64_RAT count     15.000000     25.000000  ...     43.000000      2.000000
                 mean      26.288000     25.992800  ...     19.134884     15.860000
                 std        6.256164      5.829563  ...      7.127190      0.240416
                 min       15.980000     10.910000  ...      5.440000     15.690000
                 25%       23.155000     22.360000  ...     13.410000     15.775000
                 50%       25.460000     26.600000  ...     19.100000     15.860000
                 75%       30.070000     29.590000  ...     22.165000     15.945000
                 max       36.320000     35.530000  ...     38.250000     16.030000
sal_mean         count     15.000000     25.000000  ...     43.000000      2.000000
                 mean    1250.466667    913.360000  ...   1242.441860    769.500000
                 std      265.233285    102.925896  ...    474.272240     48.790368
                 min      796.000000    681.000000  ...    702.000000    735.000000
                 25%     1028.500000    862.000000  ...    916.000000    752.250000
                 50%     1218.000000    916.000000  ...   1087.000000    769.500000
                 75%     1477.500000    956.000000  ...   1522.000000    786.750000
                 max     1652.000000   1181.000000  ...   2606.000000    804.000000
factory_num      count     15.000000     25.000000  ...     43.000000      2.000000
                 mean       5.866667      9.360000  ...      9.232558    133.000000
                 std       10.795943     13.369368  ...     11.552854     11.313708
                 min        0.000000      0.000000  ...      0.000000    125.000000
                 25%        1.000000      2.000000  ...      1.000000    129.000000
                 50%        2.000000      5.000000  ...      5.000000    133.000000
                 75%        6.500000     13.000000  